In [3]:
import pandas as pd
import numpy as np
import jdatetime as jdt
import requests
from collections import namedtuple
from itertools import combinations
from operator import add
import re 

In [ ]:
def day_to_ex(ex_date, hist_date):
            exd = ex_date.split("-")
            hd = hist_date.split("-")
            t = jdt.date(int(exd[0]), int(exd[1]), int(exd[2])) - jdt.date(
                int(hd[0]), int(hd[1]), int(hd[2])
            )

In [5]:
def covered_call(df:pd.DataFrame):
    """
    خرید داراییِ پایه و فروشِ همزمانِ اختیارِ خرید
    سود-زیان برابر ارزشِ فروش اختیار و تفاوت ارزشِ روزِ دارایی ( در صورت افزایش تا حد قیمتِ اعمال) با ارزشِ خریدِ دارایی است.

    :param ua_price:
    :param premium:
    :param strike_price:
    :return: max_pot_profit: حداکثر سود بالقوه- در شرایطی ایجاد می‌شود که قیمتِ داراییِ پایه در تاریخ سر-رسید از قیمتِ اعمال بالاتر-برابر باشد
            break_even : اگر قیمتِ داراییِ پایه از این قیمت پایین-تر بیاد، معامله ،وارد زیان می‌شود
            pct_max_profit :
            current_profit : بر اساسِ قیمتِ کنونیِ داراییِ پایه محاسبه می‌شود
            pct_current_profit :
    """
    stg = namedtuple("Strategy", "buy sell")
    df["covered_call"] = df.apply(lambda x: stg(buy=x["ua"], sell="ticker"))
    df = df.assign(max_pot_profit=df.strike_price - df.ua_sell_price + df.buy_price)
    df = df.assign(break_even=df.ua_sell_price - df.buy_price)
    df = df.assign(pct_max_profit=df.max_pot_profit / df.break_even * 100)
    df["current_profit"] = df.apply(
        lambda x: min(x["strike_price"], x["ua_sell_price"]) - x["ua_sell_price"] + x["buy_price"], axis=1)
    df = df.assign(pct_current_profit=df.current_profit / df.break_even * 100)  

    return df

In [ ]:
def call_back_spread(self, df, times):
    stg = namedtuple("Strategy", "buy sell")
    groups = df.groupby(by=["ticker", "t"])
    df_ = pd.DataFrame()
    for name, group in groups:
        group.reset_index(inplace=True)
        group = group.sort_values(by=["strike_price"], ascending=False)
        if len(group) > 1:
            combo_o_ticker = list(stg(buy=b, sell=s) for b, s in combinations(group.o_ticker, 2))
            combo_strike_price = list(stg(buy=b, sell=s) for b, s in combinations(group.strike_price, 2))
            combo_o_adj_final = list(stg(buy=b, sell=s) for b, s in combinations(group.o_adj_final, 2))
            combo_bs = list(stg(buy=b, sell=s) for b, s in combinations(group.bs, 2))
            premium = [-pb * times + ps for pb, ps in combo_o_adj_final]
            max_pot_loss = list(map(add, [-sb + ss for sb, ss in combo_strike_price], premium))
            current_profit = []
            for i in range(len(combo_strike_price)):
                if all(s >= group.adj_final.values[0] for s in combo_strike_price[i]):
                    current_profit.append(premium[i])
                elif combo_strike_price[i].buy >= group.adj_final.values[0]:
                    current_profit.append(- group.adj_final.values[0] + combo_strike_price[i].sell + premium[i])
                else:
                    current_profit.append((- group.adj_final.values[0] + combo_strike_price[i].sell) + (
                                group.adj_final.values[0] - combo_strike_price[i].buy) * times + premium[i])

            df_group = pd.DataFrame({
                "o_ticker": combo_o_ticker,
                "strike_price": combo_strike_price,
                "t": group.t.values[0],
                "bs": combo_bs,
                "adj_final": group.adj_final.values[0],
                "o_adj_final": combo_o_adj_final,
                "max_pot_loss": max_pot_loss,
                "current_profit": current_profit,
            })
            df_ = pd.concat([df_, df_group])
    return df_

In [4]:
stg = namedtuple("Strategy", "buy sell")
stg("a", "d")

Strategy(buy='a', sell='d')